This tutorial and the assets can be downloaded as part of the [Wallaroo Tutorials repository](https://github.com/WallarooLabs/Wallaroo_Tutorials/blob/wallaroo2024.1_tutorials/wallaroo-model-deploy-and-serve/xgboost-upload-tutorials).

## Booster RF Regression Example

The following tutorial demonstrates deploying and serving an Booster RF Regression model to Wallaroo.

The following XGBoost model types are supported by Wallaroo.  XGBoost models **not** supported by Wallaroo are supported via the [Arbitrary Python models, also known as Bring Your Own Predict (BYOP)](https://staging.docs.wallaroo.ai/wallaroo-model-operations/wallaroo-model-operations-deploy/wallaroo-model-operations-upload-register/wallaroo-model-operations-upload-register-arbitrary-python/).

| XGBoost Model Type | Wallaroo Auto Packaging Supported |
|---|---|
| XGBClassifier | √ |
| XGBRegressor | √ |
| Booster Classifier | √ |
| Booster Classifier | √ |
| Booster Regressor | √ |
| Booster Random Forest Regressor | √ |
| Booster Random Forest Classifier | √ |
| XGBRFClassifier | √ |
| XGBRFRegressor | √ |
| XGBRanker\* | X |

* XGBRanker XGBoost models are currently supported via converting them to [BYOP](https://staging.docs.wallaroo.ai/wallaroo-model-operations/wallaroo-model-operations-deploy/wallaroo-model-operations-upload-register/wallaroo-model-operations-upload-register-arbitrary-python/) models.

## Goal

Upload, deploy, and serve a sample Booster RF Regression model.

### Resources

This tutorial provides the following:

* Models:
  * `./models/xgb_booster_rf_regression.pkl`: The sample XGBoost model that receives the `sklearn.datasets.load_diabetes` dataset.

### Prerequisites

* A deployed Wallaroo instance with [Edge Registry Services](https://docs.wallaroo.ai/wallaroo-operations-guide/wallaroo-configuration/wallaroo-edge-deployment/#enable-wallaroo-edge-deployment-registry) and [Edge Observability enabled](https://docs.wallaroo.ai/wallaroo-operations-guide/wallaroo-configuration/wallaroo-edge-deployment/#set-edge-observability-service).
* The following Python libraries installed:
  * [`wallaroo`](https://pypi.org/project/wallaroo/): The Wallaroo SDK. Included with the Wallaroo JupyterHub service by default.
  * [`pandas`](https://pypi.org/project/pandas/): Pandas, mainly used for Pandas DataFrame
* A X64 Docker deployment to deploy the model on an edge location.
* The notebook "Wallaroo Run Anywhere Model Drift Observability with Assays: Preparation" has been run, and the model edge deployments executed.

## Steps

### Import Libraries

The first step is to import the libraries we will need.  See `./requirements.txt` for a list of additional libraries used with this tutorial.

In [1]:
import wallaroo
from wallaroo.pipeline import Pipeline
from wallaroo.deployment_config import DeploymentConfigBuilder
import pyarrow as pa
from wallaroo.framework import Framework

import pickle
from sklearn.datasets import load_diabetes
from xgboost import train, DMatrix
from sklearn.model_selection import train_test_split

### Open a Connection to Wallaroo

The next step is connect to Wallaroo through the Wallaroo client.  The Python library is included in the Wallaroo install and available through the Jupyter Hub interface provided with your Wallaroo environment.

This is accomplished using the `wallaroo.Client()` command, which provides a URL to grant the SDK permission to your specific Wallaroo environment.  When displayed, enter the URL into a browser and confirm permissions.  Store the connection into a variable that can be referenced later.

If logging into the Wallaroo instance through the internal JupyterHub service, use `wl = wallaroo.Client()`.  If logging in externally, update the `wallarooPrefix` and `wallarooSuffix` variables with the proper DNS information.  For more information on Wallaroo DNS settings, see the [Wallaroo DNS Integration Guide](https://docs.wallaroo.ai/wallaroo-operations-guide/wallaroo-configuration/wallaroo-dns-guide/).

In [2]:
wl = wallaroo.Client()

### Set Variables

We'll set the name of our workspace, pipeline, models and files.  Workspace names must be unique across the Wallaroo workspace.  For this, we'll add in a randomly generated 4 characters to the workspace name to prevent collisions with other users' workspaces.  If running this tutorial, we recommend hard coding the workspace name so it will function in the same workspace each time it's run.



In [3]:
workspace_name = f'booster-rf-regression'
pipeline_name = f'booster-rf-regression'

model_name = 'booster-rf-regression'
model_file_name = './models/xgb_booster_rf_regression.pkl'

### Create Workspace and Pipeline

We will now create the Wallaroo workspace to store our model and set it as the current workspace.  Future commands will default to this workspace for pipeline creation, model uploads, etc.  We'll create our Wallaroo pipeline to deploy our model.

In [4]:
workspace = wl.get_workspace(name=workspace_name, create_if_not_exist=True)
wl.set_current_workspace(workspace)

pipeline = wl.build_pipeline(pipeline_name)

### Upload XGBoost Model

XGBoost models are uploaded to Wallaroo through the `wallaroo.client.Client.upload_model` method.

#### Upload XGBoost Model Parameters

The following parameters are available for XGBoost models.  

| Parameter | Type | Description |
|---|---|---|
|`name` | `string` (*Required*) | The name of the model.  Model names are unique per workspace.  Models that are uploaded with the same name are assigned as a new **version** of the model. |
|`path` | `string` (*Required*) | The path to the model file being uploaded. 
|`framework` |`string` (*Required*) | Set as `Framework.XGBOOST`. |
|`input_schema` | `pyarrow.lib.Schema` (*Required*) | The input schema in Apache Arrow schema format. |
|`output_schema` | `pyarrow.lib.Schema` (*Required*) | The output schema in Apache Arrow schema format. |
| `convert_wait` | `bool` (*Optional*) (*Default: True*) | <ul><li>**True**: Waits in the script for the model conversion completion.</li><li>**False**:  Proceeds with the script without waiting for the model conversion process to display complete. |

Once the upload process starts, the model is containerized by the Wallaroo instance.  This process may take up to 10 minutes.

#### Upload XGBoost Model Return

The following is returned with a successful model upload and conversion.

| Field | Type | Description |
|---|---|---|
| `name` | string | The name of the model. |
| `version` | string | The model version as a unique UUID. |
| `file_name` | string | The file name of the model as stored in Wallaroo. |
| `image_path` | string | The image used to deploy the model in the Wallaroo engine. |
| `last_update_time` | DateTime | When the model was last updated. |#

### Configure Input and Output Schemas

First we configure the input and output schemas in PyArrow format.

In [5]:
input_schema = pa.schema([
    pa.field('inputs', pa.list_(pa.float32(), list_size=10))
])

output_schema = pa.schema([
    pa.field('predictions', pa.float32()),
])

### Upload Model Example

With the input and output schemas defined, we now upload the XGBoost model.

In [6]:
model = wl.upload_model(model_name, 
                        model_file_name, 
                        framework=Framework.XGBOOST, 
                        input_schema=input_schema, 
                        output_schema=output_schema)
model

Waiting for model loading - this will take up to 10.0min.
Model is pending loading to a native runtime.
Model is attempting loading to a native runtime.successful

Ready


Name,booster-rf-regression
Version,436f63ea-47ca-4921-9ee5-dcef948a843d
File Name,xgb_booster_rf_regression.pkl
SHA,b58b410a1eb4690dcf1bdcd08157f37253d8316cafd406a165b484ceb47408b3
Status,ready
Image Path,None
Architecture,x86
Acceleration,none
Updated At,2024-11-Apr 20:34:17


### Deploy Pipeline

With the model uploaded and packaged, we add the model as a pipeline model step.  For our deployment, we will set a minimum deployment configuration - this is the amount of resources the deployed pipeline uses from the cluster.

Once set, we deploy the pipeline, which allocates the assigned resources for the cluster and makes it available for inference requests.

In [7]:
pipeline.add_model_step(model)

deployment_config = DeploymentConfigBuilder() \
    .cpus(0.25).memory('1Gi') \
    .build()

In [8]:
pipeline.deploy(deployment_config=deployment_config)
pipeline.status()

{'status': 'Running',
 'details': [],
 'engines': [{'ip': '10.28.0.164',
   'name': 'engine-7bf7dfb769-rl7h9',
   'status': 'Running',
   'reason': None,
   'details': [],
   'pipeline_statuses': {'pipelines': [{'id': 'booster-rf-regression',
      'status': 'Running'}]},
   'model_statuses': {'models': [{'name': 'booster-rf-regression',
      'sha': 'b58b410a1eb4690dcf1bdcd08157f37253d8316cafd406a165b484ceb47408b3',
      'status': 'Running',
      'version': '436f63ea-47ca-4921-9ee5-dcef948a843d'}]}}],
 'engine_lbs': [{'ip': '10.28.1.127',
   'name': 'engine-lb-d7cc8fc9c-xnb2r',
   'status': 'Running',
   'reason': None,
   'details': []}],
 'sidekicks': []}

### Run Sample Inference

The dataset is from the `sklearn.datasets.load_breast_cancer` examples.  These are converted to a pandas DataFrame, that is submitted to the deployed model in Wallaroo for an inference request.

In [9]:
dataset = load_diabetes()

# assuming the model is trained on the following DMatrix
X, y = dataset.data, dataset.target
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
dtrain = DMatrix(X_train, label=y_train)
dtest = DMatrix(X_test, label=y_test)

data = dtest.get_data().todense()[:100]

In [10]:
import pandas as pd

dataframe = pd.DataFrame({"inputs": data.tolist()})

In [11]:
results = pipeline.infer(dataframe)
results

,time,in.inputs,out.predictions,anomaly.count
0,2024-04-11 20:34:34.359,"[0.0453409851, -0.0446416363, -0.0062059541, -...",154.345660,0
1,2024-04-11 20:34:34.359,"[0.0925639868, -0.0446416363, 0.0369065292, 0....",161.982850,0
2,2024-04-11 20:34:34.359,"[0.0635036752, 0.0506801195, -0.0040503298, -0...",154.345660,0
3,2024-04-11 20:34:34.359,"[0.0961965248, -0.0446416363, 0.0519958995, 0....",244.706510,0
4,2024-04-11 20:34:34.359,"[0.0126481373, 0.0506801195, -0.0202175118, -0...",108.613180,0
...,...,...,...,...
84,2024-04-11 20:34:34.359,"[0.0017505219, -0.0446416363, -0.065485619, -0...",104.921776,0
85,2024-04-11 20:34:34.359,"[0.0126481373, -0.0446416363, -0.0256065708, -...",78.300160,0
86,2024-04-11 20:34:34.359,"[-0.0273097865, -0.0446416363, -0.0633299947, ...",82.259480,0
87,2024-04-11 20:34:34.359,"[-0.0236772466, -0.0446416363, -0.0697968677, ...",68.962540,0


### Undeploy the Pipeline

With the tutorial complete, we undeploy the pipeline and return the resources back to the cluster.

In [12]:
pipeline.undeploy()

name,booster-rf-regression
created,2024-04-11 20:34:07.909749+00:00
last_updated,2024-04-11 20:34:20.805478+00:00
deployed,False
arch,x86
accel,none
tags,
versions,"535c2ef2-4ed9-414d-8f21-188086b7acbd, 165b59e5-9b64-4ba7-a2f1-b565da86a74b"
steps,booster-rf-regression
published,False
